Reproducible steps to create a corpus from [EDGAR](https://www.sec.gov/edgar) 10-k filings.

# Example

Retrieve a single file by hand.

1. Navigate to [here](https://www.sec.gov/edgar/searchedgar/companysearch)
2. Enter "ENV" into the search box.
3. Right hand side, expand "10-K (annual reports) and 10-Q (quarterly reports)"
4. Get whatever is on top.

As of 2024/01/03

"ENV" is:

* https://www.sec.gov/edgar/browse/?CIK=1337619
* https://www.sec.gov/ix?doc=/Archives/edgar/data/1337619/000133761923000012/env-20221231.htm

"MSFT" is:

* https://www.sec.gov/edgar/browse/?CIK=789019
* https://www.sec.gov/ix?doc=/Archives/edgar/data/789019/000095017023035122/msft-20230630.htm


# Pseudocode

Below is a list of the steps we take.
Keep in mind that these steps are a 10 thousand foot view.
The implementation will be commented to a more detailed level.

1. Get the tickers from the [SEC](https://www.sec.gov/file/company-tickers)
2. Using the retrieved data, get the accession documents for the 10-Ks (`form_type`) the past 20 (`limit`) years
3. Using the retrieved data, get the XHTML files

You can force a full re-download by deleting everything in _~/data/10-k_.
Otherwise the script will do a checkpoint evaluation of how far it has processed.
It will skip steps it thinks are done.

In [ ]:
from pathlib import Path

tickers_url = 'https://www.sec.gov/files/company_tickers.json'
user_agent = 'TextCorpusLabs/EDGAR'
limit = 20
form_type = '10-K'

data_folder = Path(f'./data/{form_type}')
tickers_file = data_folder.joinpath('./tickers.csv')
accessions_file = data_folder.joinpath('./accession.metadata.csv')

# Step 1

1. Get the list of tickers from the SEC
2. Convert the tickers into an array, then sort it.
3. Save the tickers to a CSV

In [ ]:
import requests
import pandas as pd
import json

def get_tickers(tickers_file: Path, tickers_url: str, user_agent: str, ) -> pd.DataFrame:
    if not tickers_file.exists():
        tickers = None
        with requests.Session() as session:
            session.headers['User-Agent'] = user_agent
            with session.get(tickers_url) as result:
                if result.status_code == 200:
                    t1 = json.loads(result.text)
                    t2 = [x for x in t1.values()]
                    t3 = sorted(t2, key = lambda tup: tup['ticker'])
                    tickers = [(x['cik_str'], x['ticker'], x['title']) for x in t3]
        if tickers is not None:
            df = pd.DataFrame(tickers, columns = ['CIK', 'Ticker', 'Name'])
            if not tickers_file.parent.exists():
                tickers_file.parent.mkdir(parents = True)
            df.to_csv(tickers_file, index = False)
        else:
            raise RuntimeError('Error retrieving tickers')          
    return pd.read_csv(tickers_file) #type: ignore

tickers = get_tickers(tickers_file, tickers_url, user_agent)

# Step 2

For each CIK in _tickers.csv_ (Step 1)

1. Get the accessions for the past 20 10-Ks

Save all the accessions for all the CIKs to disk

**Note 1**: Notice `tickers.CIK.unique()`.
The data pull needs to be done on CIK, not ticker.
A single company can have more than one ticker (AACI vs AACIU), byt only one CIK (1844817).

**Note 2**: Notice `except ValueError: pass`.
It is possible for a CIK (or ticker) to have no associated documents of a particular type(10-k).
`get_filing_metadatas()` responds to this case by throwing an error.
On our side, it just means skip the record.


In [ ]:
#type: ignore
#cSpell: ignore tqdm, metadatas, dtype
from sec_downloader import Downloader
from tqdm.notebook import tqdm
import sec_downloader.types as sec_t
import typing as t

def get_accession_metadata(accessions_file: Path, tickers: pd.DataFrame, form_type: str, limit: int, user_agent: str) -> pd.DataFrame:
    if not accessions_file.exists():
        metadata: t.List[sec_t.FilingMetadata] = []
        downloader = Downloader(user_agent, '')
        for cik in tqdm(tickers.CIK.unique()):
            try:
                t1 = downloader.get_filing_metadatas(sec_t.RequestedFilings(ticker_or_cik  = cik, form_type = form_type, limit = limit))
                metadata.extend(t1)
            except ValueError:
                pass
        if len(metadata) > 0:
            df = pd.DataFrame(metadata)
            df = df[["cik", "accession_number", "report_date", "primary_doc_url"]]
            df = df.rename(columns={'cik': 'CIK', 'accession_number': 'Accession Number', 'report_date': 'Report Date', 'primary_doc_url': 'URL'})
            if not accessions_file.parent.exists():
                accessions_file.parent.mkdir(parents = True)
            df.to_csv(accessions_file, index = False)
        else:
            raise RuntimeError('Error retrieving accessions')
    return pd.read_csv(accessions_file, dtype = {'CIK': int, 'Accession Number': str, 'Report Date': str, 'URL': str}, parse_dates = ['Report Date'])

accessions = get_accession_metadata(accessions_file, tickers, form_type, limit, user_agent)

# Step 3

For each accession in _accessions.csv_ (Step 2)

1. Get the XHTML document
2. save it to disk as _~/data/10-k/{year}/{cik}.{accession number}.xhtml_

In [ ]:
#cSpell: ignore tqdm
#type: ignore
from datetime import datetime
from dataclasses import dataclass

def get_accession_metadata(data_folder: Path, accessions: pd.DataFrame, user_agent: str) -> None:
    @dataclass
    class Accession:
        CIK: int
        AccessionNumber: str
        ReportDate: datetime
        URL: str
        def __init__(self, record: t.Dict[str, t.Union[int, str, datetime]]):
            self.CIK = record['CIK']
            self.AccessionNumber = record['Accession Number']
            self.ReportDate = record['Report Date']
            self.URL = record['URL']
    def get_filing_xhtml(session: requests.Session, accession: Accession) -> t.Union[None, str]:
        with session.get(accession.URL) as response:
            if response.status_code == 200:
                return response.text
        return None
    def get_xhtml_file_path(data_folder: Path, accession: Accession) -> Path:
        year = str(accession.ReportDate.year)
        year = year if year != 'nan' else '0000'
        return data_folder.joinpath(f'{year}/{accession.CIK}.{accession.AccessionNumber}.xhtml')
    with requests.Session() as session:
        session.headers['User-Agent'] = user_agent
        for accession in tqdm([Accession(x) for x in accessions.to_dict('records')]): 
            xhtml_file = get_xhtml_file_path(data_folder, accession)
            if not xhtml_file.parent.exists():
                xhtml_file.parent.mkdir(parents = True)
            if xhtml_file.exists():
                continue
            xhtml = get_filing_xhtml(session, accession)
            if xhtml is None:
                print(f'{accession.CIK}.{accession.AccessionNumber} failed')
            else:
                with open(xhtml_file, mode = 'w') as fp:
                    fp.write(xhtml)

get_accession_metadata(data_folder, accessions, user_agent)